# Scraping: récupération de la page d'accueil du journal Le Soir

Dans ce notebook, nous créons un robot qui va ouvrir la page d'accueil du site du journal [Le Soir](https://www.lesoir.be/) et récupérer le titre de tous les articles du jour et les stocker dans un fichier csv.

## Imports

In [1]:
import os
import re
import time
import requests
from bs4 import BeautifulSoup
import pandas as pd

## Récupération de tous les articles de la page d'accueil



In [2]:
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'}

articles = []

root_url = f"https://www.lesoir.be"
response = requests.get(root_url, headers=headers)
soup = BeautifulSoup(response.text, 'html.parser')
for link in soup.find_all("a", {"class":"r-article--link"}):
    title = link.text.strip()
    url = root_url + link.get('href')
    articles.append([url, title])

In [3]:
# Affichage du nombre d'articles récupérés
len(articles)

88

In [4]:
# Affichage des 10 premières entrées
articles[:10]

[['https://www.lesoir.be/482009/article/2022-12-08/paul-magnette-charge-le-greffier-du-parlement-wallon-et-epargne-marcourt',
  'Paul Magnette charge le greffier du parlement wallon et épargne Marcourt'],
 ['https://www.lesoir.be/482005/article/2022-12-08/parlement-wallon-jean-claude-marcourt-ne-vacille-pas-sur-son-siege-de-president',
  'Parlement wallon: Jean-Claude Marcourt ne vacille pas sur son siège de président'],
 ['https://www.lesoir.be/481979/article/2022-12-08/temperatures-glaciales-8-degres-attendus-ces-prochaines-heures-les-previsions',
  'Températures glaciales: -8 degrés attendus ces prochaines heures, les prévisions météo région par région'],
 ['https://www.lesoir.be/481974/article/2022-12-08/proces-des-attentats-de-bruxelles-la-question-du-traitement-des-accuses-plombe',
  'Procès des attentats de Bruxelles: la question du traitement des accusés plombe l’audience pour de bon'],
 ['https://www.lesoir.be/481962/article/2022-12-08/les-temoignages-glacants-de-8-jeunes-femm

## Création d'un dataframe avec les liens et les titres des articles


In [5]:
df = pd.DataFrame(articles, columns=['link', 'title'])
df

,link,title
0,https://www.lesoir.be/482009/article/2022-12-0...,Paul Magnette charge le greffier du parlement ...
1,https://www.lesoir.be/482005/article/2022-12-0...,Parlement wallon: Jean-Claude Marcourt ne vaci...
2,https://www.lesoir.be/481979/article/2022-12-0...,Températures glaciales: -8 degrés attendus ces...
3,https://www.lesoir.be/481974/article/2022-12-0...,Procès des attentats de Bruxelles: la question...
4,https://www.lesoir.be/481962/article/2022-12-0...,Les témoignages glaçants de 8 jeunes femmes qu...
...,...,...
83,https://www.lesoir.be/480708/article/2022-12-0...,"Prime, bonus, chèque cadeau: qu’est-ce qui vou..."
84,https://www.lesoir.be/479242/article/2022-11-2...,Des frais supplémentaires pour les détenteurs ...
85,https://www.lesoir.be/478340/article/2022-11-2...,Alimentation: pourquoi les supermarchés n’en f...
86,https://www.lesoir.be/479077/article/2022-11-2...,L’épargne-pension: pourquoi c’est intéressant ...


In [6]:
# Sauvegarde du dataframe dans un fichier csv
df.to_csv(f"../data/lesoir_{time.strftime('%Y%m%d')}.csv", index=False)

## Téléchargement d'un article et affichage du texte

In [7]:
# Récupération du premier lien du dataframe
article_url = df.iloc[0][0]

# Ouverture de la page
response = requests.get(article_url, headers=headers)
soup = BeautifulSoup(response.text, 'html.parser')

# Parsing du contenu
title = soup.find("h1").get_text(separator=' ')
chapo = soup.find("r-article--chapo").get_text(separator=' ')
content = soup.find("r-article--section").get_text(separator=' ')

print(title)
print("==================================")
print(chapo)
print("==================================")
print(content)

Paul Magnette charge le greffier du parlement wallon et épargne Marcourt
Selon le président du PS, les parlementaires n’ont pas fermé les yeux en raison de l’emprise du greffier, mais ont été simplement trompés par son apparence professionnelle.

 L es voyages controversés effectués l’an passé par le greffier du parlement wallon Frédéric Janssens, accompagnés parfois de son président Jean-Claude Marcourt ont fait l’objet d’une discussion, ce jeudi, en bureau élargi de l’assemblée. 
 
 
 
 
 
 À lire aussi 
 Voyages du parlement wallon: les dépenses somptuaires de Marcourt et du greffier 
 
 
 
 
 Paul Magnette était l’invité de l’émission  Jeudi en prime , sur la RTBF. Le président du parti socialiste s’est exprimé au sujet de ces récentes révélations, indiquant : « Les responsabilités sont collectives ». Selon Paul Magnette, les responsabilités sont partagées et le problème est « général ». Le président du PS a ensuite appelé à des contrôles plus stricts que ceux en vigueur actuelleme

### Nettoyage du texte à l'aide d'expressions régulières

In [8]:
# Suppression de tout ce qui se trouve entre parenthèses
clean_content = re.sub("\([^\)]+\)", " ", content)
# Suppression des espaces multiples
clean_content = re.sub("\s+", " ", clean_content)

print(content)
print("==================================")
print(clean_content)



 L es voyages controversés effectués l’an passé par le greffier du parlement wallon Frédéric Janssens, accompagnés parfois de son président Jean-Claude Marcourt ont fait l’objet d’une discussion, ce jeudi, en bureau élargi de l’assemblée. 
 
 
 
 
 
 À lire aussi 
 Voyages du parlement wallon: les dépenses somptuaires de Marcourt et du greffier 
 
 
 
 
 Paul Magnette était l’invité de l’émission  Jeudi en prime , sur la RTBF. Le président du parti socialiste s’est exprimé au sujet de ces récentes révélations, indiquant : « Les responsabilités sont collectives ». Selon Paul Magnette, les responsabilités sont partagées et le problème est « général ». Le président du PS a ensuite appelé à des contrôles plus stricts que ceux en vigueur actuellement. 
 
                    « Il a abusé »
                    
                 
 Selon Paul Magnette, les parlementaires n’ont pas fermé les yeux en raison de l’emprise du greffier, mais ont été simplement trompés par son apparence professionnel

### Création d'un fichier avec le contenu de l'article


In [9]:
with open("../data/lesoir_example.txt", "w") as writer:
    writer.write(f"{title}\n\n{chapo}\n\n{clean_content}")

## Pour en savoir plus

- Le web scraping avec Python: https://realpython.com/beautiful-soup-web-scraper-python/
- Tutoriel sur les expressions régulières: https://www.w3schools.com/python/python_regex.asp